# Advanced bash: using Coprocesses

a background process where your shell get file descriptors for the process's stdin and stdout 
- implemented with pipe

To make a coprocess: create a script that is a filter 

```
#!/bin/bash
while 
  read line
do  
  echo $line | tr "ABC" "abc" # translate command 
done
```

To use:
`coproc ./mycoproc.sh`

```
echo BANANA >&"${CROPROC[1]}"
cat <&"${COPROC[0]}"
```

### Name coprocess 
`coporc my { ./mycoproc.sh; }`
```
echo BANANA >&"${my[1]}"
cat <&"${my[0]}"
```

* starts a process in the background where we can continually pipe in data (COPROC[1]) then call ALL data that has been filtered with COPROC[0]

### Example: "translate.sh"

```
#!/bin/bash
declare -l line
while 
  read line
do
  echo $line
done
```
`coproc ./translate.sh`

```
echo BaNAna >&"${COPROC[1]}"
cat <&"${COPROC[0]}"
```




# Debugging scripts 

- `$ bash prog` .. run prog; don't need to execute permission
- `$ bash -x prog` .. echo commands after processing; can also do `set -x` (sets x in script) or `set +x` (unsets x in script) inside a script, Usefult to trace the what is occurring in script
- `$ bash -n prog` .. do not execute commands, checks for syntax only 
- `set -u` .. reports usage of an unset variable. Great for catching variable which has not been referenced before due to typos in var names 
- Lots of echo statements are helpful
- `tee` .. write output out to std output and out to file 
` cmd | tee log.file | ...`

### Example 
debug1.sh

```
#!/bin/bash
a=1
echo my PID is $$
b=2
echo a is $a b is $b c is $c
if 
true
tehn # causes syntax error
    echo true is successful
else
    echo true is not successful
fi
```

`bash -x ./debug1.sh` .. prints out lines in script before failure 
`-n` .. only prints out syntax errors and doesn't execute script

#### In process..
- `set -u` used to check for unset variables. Would fail if unset var is called
- `set -x` turn on tracing 
- `set +x` turns off tracing 



## Trap: using signals 

`bash trap` command is for signal handling 
- changes behavior of signals within a script
- ignore signals during critical sections in a script 
- allow the scripts to die gracefully

Works by... giving a string of stuff to do then name a signal to do it to 
- `INT` is interrupt signal ctrl+c
- `QUIT` is quit signal ctrl+\ 

```
cat trapit.sh
trap 'echo you just got int; exit' INT
trap "echo you cannot quit now" QUIT

cd / 
while
true
do
    echo looping
    du -m * 2>/dev/null
    echo sleeping
    sleep 5
done
```

with ctrl+\, the shell won't quit just processes since there is no `; exit`

- `trap "" QUIT` makes it so ctrl+/ does not work, is ignored 
- can nest traps within a tral 







## The Eval command 
- used to have bash evaluate a string 
- makes a "second pass" over the string and then runs it as a command 
- runs "data," in effect, so be carful about providng a way for arbitrary code execution

`c="ls | tr 'a' 'A' ; $c` doesn't work but `; eval $c` would 
- possible security risk as `eval` would run any script in variables


## The Getopt command
- `getopt` is used to process command-line options

`opts='getopt -o a: -l apple -- "$@"'`
- option `a`, long form `apple`, takes an argument 
- `getopt` prints the parsed arguments
- `$@` is the bash variable for list of parameters passed
- `--` used to mark end of processing 

- it is typical to loop through the arguments using a case statement to match and handle them 



### Challenges 
- write a bash script that assigns to a  variable c, the value "ls -s | sort -n"
- run just by $c 
- run with eval $c

```
#!/bin/bash

c='ls -s | sort -n'
```
What happens?
- if variable has pipe or function. must invoke with `eval` for bash to understand and execute


- write a bash script which sets a variable, opts equal to "a b \$1 \$2"
- do `set -- "opts"` and echo $@. Run the script with command line options x y z. Do you see x and y?
- change `set -- "$opts"` to `eval set -- "$opts"` and run again 

``` 
cat challenge2.sh
opts='a b \$1 \$2'
set -- "opts"
echo $@
-----
eval set -- $opts
echo $@

challenge2.sh x y z
```
Take away: eval used in set to process args of a command in shell script

- Create a coproc that will run a script that uses `sed` to use the substitue command to replace every occurrence of flag with banner 

```
cat coproc-sed
while
read line
do 
    echo $line | sed 's/flag/banner/g'
done
```

```
coproc mysed { ./coproc-sed ; } #renames and executes in background
echo ${mysed[1]} #58 is output file descriptor, write to 58
echo ${mysed[0]} #62 is where we read from

cat coproc-data >&58 #send stdout to 58 (coproc) says read to 58 
cat <&62 #reads data file 
```
By specifying where file is read and write to we pass coproc-data to the coproc shell script executing

